# Unit 5 - Financial Planning

In [ ]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
# Load .env enviroment variables
load_dotenv()

In [ ]:
api_key = os.getenv('ALPACA_API_KEY')
secret_key = os.getenv('ALPACA_SECRET_KEY')
print(api_key)
print(secret_key)

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [ ]:
btc_data = requests.get(btc_url).json()

eth_data = requests.get(eth_url).json()
btc_data

In [ ]:
eth_cad_price = eth_data['data']['1027']['quotes']['CAD']['price']
eth_cad_price

In [ ]:

# Fetch current BTC price
btc_cad_price = btc_data['data']['1']['quotes']['CAD']['price']
btc_cad_price

# Fetch current ETH price
eth_cad_price = eth_data['data']['1027']['quotes']['CAD']['price']
eth_cad_price

# Compute current value of my crpto

crpyto = btc_cad_price + eth_cad_price
crpyto

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${btc_cad_price:0.2f}")
print(f"The current value of your {my_eth} ETH is ${eth_cad_price:0.2f}")
print(f"The cuurent value of my crypto is ${crpyto:0.2f}")

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object

# api = tradeapi.REST(
#     alpaca_api_key,
#     alpaca_secret_key,
#     api_version = "v2"
# )

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2')

In [ ]:
# Format current date as ISO format
current_date = pd.Timestamp("2020-08-01", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
closing_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()

df_ticker = alpaca.get_bars(
    tickers,
    timeframe,
    start=current_date,
    end=closing_date
).df
df_ticker.tail()
# Reorganize the DataFrame
# Separate ticker data

AGG = df_ticker[df_ticker['symbol']=='AGG'].drop('symbol', axis=1)
SPY = df_ticker[df_ticker['symbol']=='SPY'].drop('symbol', axis=1)


# Concatenate the ticker DataFrames

df_ticker = pd.concat([AGG, SPY], axis=1, keys=["AGG", "SPY"])
# Preview DataFrame
df_ticker.head()

In [ ]:
# Pick AGG and SPY close prices

agg_close_price = float(df_ticker["AGG"]["close"][-1])
spy_close_price = float(df_ticker["SPY"]["close"][-1])

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# Compute the current value of shares
agg_current_price = float(df_ticker["AGG"]["close"][-1]) * my_agg
spy_current_price = float(df_ticker["SPY"]["close"][-1]) * my_spy


# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${spy_current_price:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${agg_current_price:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
monthly_income = 12000

# Consolidate financial assets data

consolidate = [btc_cad_price + eth_cad_price, agg_current_price + spy_current_price]
display(crpyto)


# Create savings DataFrame
df_savings = pd.DataFrame(consolidate, columns = ['amount'], index=['crypto','shares'])

# Display savings DataFrame
df_savings.head()

In [ ]:
# Plot savings pie chart
df_savings.plot(kind ='pie', subplots=True, title = ('Composition of Personal Savings')) 

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
total_savings = (btc_cad_price + eth_cad_price + agg_current_price + spy_current_price)

# Validate saving health
if total_savings > emergency_fund:
    print(f"Congratulations! You have enough money in your emergency fund")
elif total_savings == emergency_fund:
    print(f"Congratulations! You reached your Financial Goal")
else:
    print(f"You need reached your Financial Goal")

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2020-08-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2024-01-12', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
# YOUR CODE HERE!

df_stock_data = alpaca.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df
df_stock_data.head()
# # Set the ticker information
tickers = ["AGG", "SPY"]


# Reorganize the DataFrame

AGG = df_stock_data[df_stock_data['symbol'] == 'AGG'].drop('symbol', axis = 1)
SPY = df_stock_data[df_stock_data['symbol'] == 'SPY'].drop('symbol', axis = 1)

# Concatenate the ticker DataFrames
#variables that you are concating you can put it in a single [list type]
#parameter - 3, df, axis, keys
#[AGG, SPY] = df created
#['AGG', 'SPY'] = refers the the data in ticker symbol
df_stock_data = pd.concat([AGG, SPY], axis = 1, keys = ['AGG', 'SPY'])


# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
MC_30 = MCSimulation(portfolio_data = df_stock_data, 
                    weights = [.40,.60],
                    num_simulation = 500,
                    num_trading_days=252 * 30)
 
    


In [ ]:
# Printing the simulation input data
MC_30.portfolio_data.head() 


In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
MC_30.calc_cumulative_return() 


In [ ]:
# Plot simulation outcomes
MC_30.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
MC_30.plot_distribution()

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
MC_30_sum = MC_30.summarize_cumulative_return()

# Print summary statistics
print(MC_30_sum)

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower = round(MC_30_sum[8] * initial_investment,2)
ci_upper = round(MC_30_sum[9] * initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
     f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
ci_lower = round(MC_30_sum[8] * initial_investment,2)
ci_upper = round(MC_30_sum[9] * initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5 = MCSimulation(portfolio_data = df_stock_data, 
                    weights = [.40,.60],
                    num_simulation = 500,
                    num_trading_days=252 * 5)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
MC_5.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
MC_5.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
MC_5_sum = MC_5.summarize_cumulative_return()

# Print summary statistics
print(MC_5_sum)



In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_five = round(MC_5_sum[8] * initial_investment,2)
ci_upper_five = round(MC_5_sum[9] * initial_investment,2)
# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
MC_10 = MCSimulation(portfolio_data = df_stock_data, 
                    weights = [.40,.60],
                    num_simulation = 500,
                    num_trading_days=252 * 10)

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
MC_10.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
MC_10.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
MC_10.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
MC_10_sum = MC_10.summarize_cumulative_return()

# Print summary statistics
print(MC_10_sum)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
ci_lower_ten = round(MC_10_sum[8] * initial_investment,2)
ci_upper_ten = round(MC_10_sum[9] * initial_investment,2)
# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")